In [27]:
import kan
import kan.utils as ku
import torch
import torch.nn as nn
import numpy as np
from libraries import utils
from libraries import magnetization
from libraries import j1j2_functions
import numpy.random as npr
import qutip as qt
import sympy
import random

In [28]:
N = 10; J1 = 1; J2 = 0.2
hf = j1j2_functions.J1J2_hamiltonian(N, J1, J2)
eigsf = hf.eigenstates()
gsf = eigsf[1][0]

In [29]:
hz = j1j2_functions.J1J2_hamiltonian(N, J1, 0)
eigsz = hz.eigenstates()
gsz = eigsz[1][0]

In [30]:
print(eigsf[0][0])
print(gsf.dag() @ hf @ gsf)
print(eigsz[0][0])
print(gsz.dag() @ hz @ gsz)

-16.603853020409492
(-16.60385302040954+0j)
-18.06178541796817
(-18.06178541796818+0j)


In [31]:
print(gsz.dag() @ hf @ gsz) # swapping these for N=10, J2=0.4, fid=0.96 isn't actually that close to GSE
print(gsf.dag() @ hz @ gsf) # even for like 0.9997 fid there is still 0.01 error in these

(-16.537385583318414+0j)
(-17.981003770587222+0j)


In [32]:
gsf.full()

array([[ 0.00000000e+00+0.j],
       [-1.88810992e-49+0.j],
       [ 1.55973534e-17+0.j],
       ...,
       [-1.08596191e-49-0.j],
       [ 1.07996424e-49+0.j],
       [ 0.00000000e+00+0.j]])

In [33]:
gsz.full()

array([[ 0.00000000e+00+0.j],
       [ 1.26928255e-48+0.j],
       [-2.63794454e-18+0.j],
       ...,
       [ 1.20393997e-48+0.j],
       [-2.12190686e-48-0.j],
       [ 0.00000000e+00+0.j]])

In [34]:
abs(gsf.full() - gsz.full())

array([[0.00000000e+00],
       [1.45809355e-48],
       [1.82352979e-17],
       ...,
       [1.31253617e-48],
       [2.22990328e-48],
       [0.00000000e+00]])

In [35]:
qt.fidelity(gsz, gsf) 
# seems fidelity is basically 1 for any J2<0.5 N<=10

np.float64(0.9961878323456046)

In [36]:
threshold = 1e-12 # it seems for N=6, the f state is more sensitive, as both fit 1e-14 exactly but f state fails 1e-15
# this seems to be due to numerical inaccuracies though?
# N=6 and N=8 have everything pass
# N=10 seems to actually have things fail (order 1e-5) for J2=0.2
# J2=0.4 starts failing at N=8, N=10 fails pretty massively

In [37]:
statesz = []
signsz = []
for i in range(0, 2 ** N):
    val = gsz[i][0].real
    if abs(val) > threshold:
        statesz.append(i)
        signsz.append(-1 + 2 * int(val > 0))

In [38]:
pred_signsz =  [-1 + 2 * (magnetization.count_half_magnetization(i) % 2) for i in statesz]

In [39]:
print(len(statesz))
count = 0
for state, true, calc in zip(statesz, signsz, pred_signsz):
    if true != calc:
        print(f'fail {bin(state)}, val {gsz[state][0].real}, pred {calc}')
print('success')


252
success


In [40]:
statesf = []
signsf = []
for i in range(0, 2 ** N):
    val = gsf[i][0].real
    if abs(val) > threshold: 
        statesf.append(i)
        signsf.append(-1 + 2 * int(val > 0))

In [41]:
pred_signsf =  [-1 + 2 * (magnetization.count_half_magnetization(i) % 2) for i in statesf]

In [42]:
gsz[0b1111][0]

np.complex128(-1.1608659035393964e-17+0j)

In [43]:
print(len(statesf))
count = 0
for state, true, calc in zip(statesf, signsf, pred_signsf):
    if true != calc:
        count += 1
        print(f'fail {bin(state)}, val {gsf[state][0].real}, pred {calc}')
print(count)
print('success')


252
fail 0b11111, val -5.154362876348855e-05, pred 1
fail 0b111110, val 5.15436287634932e-05, pred -1
fail 0b1111100, val -5.154362876348732e-05, pred 1
fail 0b11111000, val 5.154362876348707e-05, pred -1
fail 0b111110000, val -5.154362876348749e-05, pred 1
fail 0b1000001111, val 5.1543628763487176e-05, pred -1
fail 0b1100000111, val -5.1543628763487244e-05, pred 1
fail 0b1110000011, val 5.1543628763487115e-05, pred -1
fail 0b1111000001, val -5.154362876348747e-05, pred 1
fail 0b1111100000, val 5.15436287634872e-05, pred -1
10
success


In [44]:
print(statesz)
print(statesf)

[31, 47, 55, 59, 61, 62, 79, 87, 91, 93, 94, 103, 107, 109, 110, 115, 117, 118, 121, 122, 124, 143, 151, 155, 157, 158, 167, 171, 173, 174, 179, 181, 182, 185, 186, 188, 199, 203, 205, 206, 211, 213, 214, 217, 218, 220, 227, 229, 230, 233, 234, 236, 241, 242, 244, 248, 271, 279, 283, 285, 286, 295, 299, 301, 302, 307, 309, 310, 313, 314, 316, 327, 331, 333, 334, 339, 341, 342, 345, 346, 348, 355, 357, 358, 361, 362, 364, 369, 370, 372, 376, 391, 395, 397, 398, 403, 405, 406, 409, 410, 412, 419, 421, 422, 425, 426, 428, 433, 434, 436, 440, 451, 453, 454, 457, 458, 460, 465, 466, 468, 472, 481, 482, 484, 488, 496, 527, 535, 539, 541, 542, 551, 555, 557, 558, 563, 565, 566, 569, 570, 572, 583, 587, 589, 590, 595, 597, 598, 601, 602, 604, 611, 613, 614, 617, 618, 620, 625, 626, 628, 632, 647, 651, 653, 654, 659, 661, 662, 665, 666, 668, 675, 677, 678, 681, 682, 684, 689, 690, 692, 696, 707, 709, 710, 713, 714, 716, 721, 722, 724, 728, 737, 738, 740, 744, 752, 775, 779, 781, 782, 787, 789, 

In [45]:
statesz == statesf # nonzero states up to N=10, J2=0.2 are the same, same for J2=0.4

True

In [46]:
for s in statesz:
    print(bin(s))

0b11111
0b101111
0b110111
0b111011
0b111101
0b111110
0b1001111
0b1010111
0b1011011
0b1011101
0b1011110
0b1100111
0b1101011
0b1101101
0b1101110
0b1110011
0b1110101
0b1110110
0b1111001
0b1111010
0b1111100
0b10001111
0b10010111
0b10011011
0b10011101
0b10011110
0b10100111
0b10101011
0b10101101
0b10101110
0b10110011
0b10110101
0b10110110
0b10111001
0b10111010
0b10111100
0b11000111
0b11001011
0b11001101
0b11001110
0b11010011
0b11010101
0b11010110
0b11011001
0b11011010
0b11011100
0b11100011
0b11100101
0b11100110
0b11101001
0b11101010
0b11101100
0b11110001
0b11110010
0b11110100
0b11111000
0b100001111
0b100010111
0b100011011
0b100011101
0b100011110
0b100100111
0b100101011
0b100101101
0b100101110
0b100110011
0b100110101
0b100110110
0b100111001
0b100111010
0b100111100
0b101000111
0b101001011
0b101001101
0b101001110
0b101010011
0b101010101
0b101010110
0b101011001
0b101011010
0b101011100
0b101100011
0b101100101
0b101100110
0b101101001
0b101101010
0b101101100
0b101110001
0b101110010
0b101110100
0b10